In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is the PRIMARY learning material!")

---

## Part 1: Register Basics

### GPU Register File

```
Register Hierarchy:

┌─────────────────────────────────────────┐
│  SM Register File (65536 × 32-bit)      │
├─────────────────────────────────────────┤
│  Shared among all threads on SM         │
│  ~256 regs/thread max, ~32-128 typical  │
└─────────────────────────────────────────┘

Access Speed:
  Registers:      ~1 cycle
  Shared Memory:  ~5-30 cycles
  L1 Cache:       ~30-80 cycles
  Global Memory:  ~300-800 cycles

Key: Registers are THE fastest memory!
```

### Register Spilling

```
When kernel needs more registers than available:

Compiler decision:
  1. Reduce concurrent threads (lower occupancy)
  2. Spill to local memory (slow!)

Local Memory = DRAM with caching
  - Uses same physical memory as global
  - Has L1/L2 cache, but still ~100x slower than registers
  - Automatic, transparent to programmer

Symptoms of spilling:
  - High local memory traffic in profiler
  - "spill stores" and "spill loads" in ptxas output
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile registers.cu
// registers.cu - Analyze register usage
#include <stdio.h>
#include <cuda_runtime.h>

// Kernel with moderate register usage
__global__ void lowRegKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float x = data[idx];
        data[idx] = x * x;
    }
}

// Kernel with high register usage
__global__ void highRegKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float a = data[idx];
        float b = data[idx + n];
        float c = data[idx + 2*n];
        float d = data[idx + 3*n];
        float e = data[idx + 4*n];
        float f = data[idx + 5*n];
        float g = data[idx + 6*n];
        float h = data[idx + 7*n];
        
        // Keep all variables live
        float r1 = a * b + c * d;
        float r2 = e * f + g * h;
        float r3 = a * c + b * d;
        float r4 = e * g + f * h;
        float r5 = r1 * r2 + r3 * r4;
        
        data[idx] = r5;
    }
}

// Query register usage
void getKernelAttributes(const char* name, const void* kernel) {
    cudaFuncAttributes attr;
    cudaFuncGetAttributes(&attr, kernel);
    
    printf("%s:\n", name);
    printf("  Registers per thread: %d\n", attr.numRegs);
    printf("  Shared memory: %zu bytes\n", attr.sharedSizeBytes);
    printf("  Local memory: %zu bytes\n", attr.localSizeBytes);
    printf("  Max threads per block: %d\n", attr.maxThreadsPerBlock);
    printf("\n");
}

int main() {
    getKernelAttributes("lowRegKernel", (const void*)lowRegKernel);
    getKernelAttributes("highRegKernel", (const void*)highRegKernel);
    return 0;
}

// Compile with: nvcc -Xptxas -v registers.cu
// to see register allocation details

In [ ]:
!nvcc -arch=sm_75 -o registers registers.cu
!./registers

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python/Numba - Demonstrate register impact

@cuda.jit
def low_register_kernel(data, result):
    """Low register usage kernel."""
    idx = cuda.grid(1)
    if idx < data.size:
        x = data[idx]
        result[idx] = x * x

@cuda.jit
def high_register_kernel(data, result):
    """Higher register usage kernel."""
    idx = cuda.grid(1)
    if idx < data.size:
        # Many variables kept live simultaneously
        a = data[idx]
        b = a * 2.0
        c = a * 3.0
        d = a * 4.0
        e = a * 5.0
        f = a * 6.0
        g = a * 7.0
        h = a * 8.0
        
        # Use all variables
        r1 = a + b + c + d
        r2 = e + f + g + h
        result[idx] = r1 * r2 o

In [ ]:
def compare_kernels(n=10_000_000):
    """Compare low vs high register kernels."""
    data = np.random.rand(n).astype(np.float32)
    result = np.zeros(n, dtype=np.float32)
    
    d_data = cuda.to_device(data)
    d_result = cuda.to_device(result)
    
    block = 256
    grid = (n + block - 1) // block
    
    # Warmup and benchmark low register
    low_register_kernel[grid, block](d_data, d_result)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(50):
        low_register_kernel[grid, block](d_data, d_result)
    cuda.synchronize()
    low_time = (time.perf_counter() - start) / 50 * 1000
    
    # Warmup and benchmark high register
    high_register_kernel[grid, block](d_data, d_result)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(50):
        high_register_kernel[grid, block](d_data, d_result)
    cuda.synchronize()
    high_time = (time.perf_counter() - start) / 50 * 1000
    
    print(f"Low register kernel:  {low_time:.3f} ms")
    print(f"High register kernel: {high_time:.3f} ms")
    print(f"Ratio: {high_time/low_time:.2f}x")

compare_kernels()

---

## Part 2: Launch Bounds

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile launch_bounds.cu
// launch_bounds.cu - Controlling register allocation
#include <stdio.h>
#include <cuda_runtime.h>

// Without launch bounds - compiler chooses register allocation
__global__ void unboundedKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Complex computation...
        float x = data[idx];
        float y = sinf(x) * cosf(x);
        float z = expf(-x * x);
        data[idx] = y + z;
    }
}

// With launch bounds - hint to compiler
__global__ void __launch_bounds__(256, 4)  // max 256 threads/block, min 4 blocks/SM
boundedKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float x = data[idx];
        float y = sinf(x) * cosf(x);
        float z = expf(-x * x);
        data[idx] = y + z;
    }
}

// Aggressive launch bounds for high occupancy
__global__ void __launch_bounds__(1024, 2)
highOccupancyKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

/*
__launch_bounds__(maxThreadsPerBlock, minBlocksPerMultiprocessor)

Effects:
  1. Compiler limits registers to fit minBlocks per SM
  2. May spill to local memory if needed
  3. Error if kernel launched with more threads than maxThreads

Guidelines:
  - maxThreadsPerBlock: your actual launch configuration
  - minBlocksPerMultiprocessor: target occupancy (2-4 typical)
*/

int main() {
    cudaFuncAttributes attr;
    
    cudaFuncGetAttributes(&attr, unboundedKernel);
    printf("Unbounded: %d regs, max %d threads\n", 
           attr.numRegs, attr.maxThreadsPerBlock);
    
    cudaFuncGetAttributes(&attr, boundedKernel);
    printf("Bounded:   %d regs, max %d threads\n", 
           attr.numRegs, attr.maxThreadsPerBlock);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o launch_bounds launch_bounds.cu
!./launch_bounds

In [ ]:
def explain_launch_bounds():
    """Explain launch bounds usage."""
    print("__launch_bounds__ Explained")
    print("=" * 60)
    print()
    print("Syntax: __launch_bounds__(maxThreadsPerBlock, minBlocksPerSM)")
    print()
    print("Parameters:")
    print("  maxThreadsPerBlock: Maximum threads you'll launch with")
    print("  minBlocksPerSM:     Minimum concurrent blocks per SM (optional)")
    print()
    print("What it does:")
    print("  1. Tells compiler the launch configuration")
    print("  2. Compiler allocates registers to fit minBlocks")
    print("  3. May cause spilling if too aggressive")
    print()
    print("Common patterns:")
    print("  __launch_bounds__(256)       // Just max threads")
    print("  __launch_bounds__(256, 2)    // 50% occupancy target")
    print("  __launch_bounds__(256, 4)    // 100% occupancy target")
    print("  __launch_bounds__(128, 8)    // High occupancy, small blocks")
    print()
    print("Trade-offs:")
    print("  Higher minBlocks → More occupancy, but may spill registers")
    print("  Lower minBlocks  → More registers, but less occupancy")

explain_launch_bounds()

---

## Part 3: Register Optimization Techniques

In [ ]:
def register_optimization_techniques():
    """Explain techniques to reduce register usage."""
    print("Register Optimization Techniques")
    print("=" * 60)
    print()
    print("1. VARIABLE REUSE")
    print("   Bad:  float a = x; float b = y; float c = a + b;")
    print("   Good: float a = x + y;  // One variable instead of three")
    print()
    print("2. REDUCE LIVE VARIABLES")
    print("   Bad:  Load all data, then process")
    print("   Good: Load, process, store - one at a time")
    print()
    print("3. COMPUTATION VS STORAGE")
    print("   Sometimes recomputing is faster than storing:")
    print("   Bad:  float s = sinf(x); ... use s multiple times")
    print("   Maybe: Recompute sinf(x) if register-bound")
    print()
    print("4. ARRAY INDEXING")
    print("   Local arrays often spill to local memory")
    print("   Use scalar variables when possible")
    print()
    print("5. LOOP UNROLLING")
    print("   #pragma unroll can increase register pressure")
    print("   Use #pragma unroll N for partial unrolling")
    print()
    print("6. SHARED MEMORY")
    print("   Move data from registers to shared memory")
    print("   Trade register pressure for shared memory pressure")

register_optimization_techniques()

### 🔷 CUDA C++ Optimization Examples

In [ ]:
%%writefile reg_optimization.cu
// reg_optimization.cu - Register optimization examples
#include <stdio.h>
#include <cuda_runtime.h>

// BAD: Many live variables
__global__ void badRegisters(float* a, float* b, float* c, float* d, float* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float v1 = a[idx];  // Live
        float v2 = b[idx];  // Live
        float v3 = c[idx];  // Live
        float v4 = d[idx];  // Live - 4 variables live!
        
        out[idx] = v1 + v2 + v3 + v4;
    }
}

// GOOD: Accumulate to reduce live variables
__global__ void goodRegisters(float* a, float* b, float* c, float* d, float* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float sum = a[idx];  // Only 1 variable live
        sum += b[idx];
        sum += c[idx];
        sum += d[idx];
        
        out[idx] = sum;
    }
}

// BAD: Local arrays spill
__global__ void badArray(float* data, float* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    float local[16];  // Likely spills to local memory!
    
    for (int i = 0; i < 16; i++) {
        local[i] = data[idx + i * n];
    }
    
    float sum = 0;
    for (int i = 0; i < 16; i++) {
        sum += local[i];
    }
    out[idx] = sum;
}

// GOOD: Process inline
__global__ void goodInline(float* data, float* out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    float sum = 0;
    for (int i = 0; i < 16; i++) {
        sum += data[idx + i * n];  // No local array needed
    }
    out[idx] = sum;
}

int main() {
    cudaFuncAttributes attr;
    
    cudaFuncGetAttributes(&attr, badRegisters);
    printf("badRegisters:  %d regs, %zu local bytes\n", attr.numRegs, attr.localSizeBytes);
    
    cudaFuncGetAttributes(&attr, goodRegisters);
    printf("goodRegisters: %d regs, %zu local bytes\n", attr.numRegs, attr.localSizeBytes);
    
    cudaFuncGetAttributes(&attr, badArray);
    printf("badArray:      %d regs, %zu local bytes\n", attr.numRegs, attr.localSizeBytes);
    
    cudaFuncGetAttributes(&attr, goodInline);
    printf("goodInline:    %d regs, %zu local bytes\n", attr.numRegs, attr.localSizeBytes);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o reg_optimization reg_optimization.cu
!./reg_optimization

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python demonstration of register optimization patterns

@cuda.jit
def bad_pattern(a, b, c, d, out):
    """Bad: Many live variables."""
    idx = cuda.grid(1)
    if idx < out.size:
        v1 = a[idx]
        v2 = b[idx]
        v3 = c[idx]
        v4 = d[idx]
        out[idx] = v1 + v2 + v3 + v4

@cuda.jit
def good_pattern(a, b, c, d, out):
    """Good: Accumulator pattern."""
    idx = cuda.grid(1)
    if idx < out.size:
        result = a[idx]
        result += b[idx]
        result += c[idx]
        result += d[idx]
        out[idx] = result

---

## Part 4: Detecting Register Issues

In [ ]:
def detect_register_issues():
    """How to detect register problems."""
    print("Detecting Register Issues")
    print("=" * 60)
    print()
    print("1. COMPILE-TIME ANALYSIS")
    print("   nvcc -Xptxas -v kernel.cu")
    print("   Look for:")
    print("     - 'Used XX registers'")
    print("     - 'XX bytes spill stores'")
    print("     - 'XX bytes spill loads'")
    print()
    print("2. RUNTIME QUERY")
    print("   cudaFuncGetAttributes(&attr, kernel);")
    print("   Check:")
    print("     - attr.numRegs (registers per thread)")
    print("     - attr.localSizeBytes (local memory per thread)")
    print()
    print("3. PROFILER ANALYSIS")
    print("   Nsight Compute metrics:")
    print("     - launch__registers_per_thread")
    print("     - lts__t_sectors_srcunit_tex_op_read (local loads)")
    print("     - lts__t_sectors_srcunit_tex_op_write (local stores)")
    print()
    print("4. OCCUPANCY CALCULATOR")
    print("   Use CUDA Occupancy Calculator spreadsheet")
    print("   Input register count, see occupancy impact")

detect_register_issues()

---

## Exercises

### Exercise 1: Reduce Register Pressure

In [ ]:
# TODO: Optimize this kernel to reduce register usage

@cuda.jit
def unoptimized_kernel(data, coeffs, result):
    """Polynomial evaluation - can you reduce registers?"""
    idx = cuda.grid(1)
    if idx < result.size:
        x = data[idx]
        c0 = coeffs[0]
        c1 = coeffs[1]
        c2 = coeffs[2]
        c3 = coeffs[3]
        c4 = coeffs[4]
        c5 = coeffs[5]
        
        # Evaluate polynomial: c0 + c1*x + c2*x^2 + ...
        x2 = x * x
        x3 = x2 * x
        x4 = x3 * x
        x5 = x4 * x
        
        result[idx] = c0 + c1*x + c2*x2 + c3*x3 + c4*x4 + c5*x5

@cuda.jit
def optimized_kernel(data, coeffs, result):
    """Your optimized version - use Horner's method!"""
    idx = cuda.grid(1)
    if idx < result.size:
        # TODO: Use Horner's method: c5 + x*(c4 + x*(c3 + x*(c2 + x*(c1 + x*c0))))
        pass

---

## Summary

### Register Hierarchy

| Level | Speed | Size |
|-------|-------|------|
| Registers | ~1 cycle | 255 per thread |
| Local Memory | ~100+ cycles | Unlimited (spill) |

### Launch Bounds

```cpp
__global__ void __launch_bounds__(maxThreads, minBlocks)
kernel(...) {
    // Compiler allocates registers to fit minBlocks per SM
}
```

### Optimization Checklist

1. ✓ Check register count with nvcc -Xptxas -v
2. ✓ Look for spill loads/stores
3. ✓ Reduce live variables
4. ✓ Use accumulator patterns
5. ✓ Avoid local arrays when possible
6. ✓ Consider launch bounds

### Tomorrow: Cache Optimization
We'll explore L1/L2 cache and memory access patterns.